In [ ]:
# Usa un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Tomado y modificado a partir de:
# https://www.datacamp.com/community/tutorials/xgboost-in-python
# https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

import pandas as pd
import numpy as np

import xgboost as xgb

from joblib import load

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV